# LangGraph Advanced Concepts: Middleware & Human-in-the-Loop (TypeScript)

Welcome to LangGraph Advanced Concepts! This notebook builds on the foundations from LangGraph 101 and introduces two powerful patterns for production agents.

**What you'll learn:**
- **Human-in-the-Loop** - Pause agents for human review and approval
- **Middleware** - Modify agent behavior at key points in execution
- **Tool Review** - Add approval workflows to sensitive tools
- **Dynamic Behavior** - Adapt agent responses based on context

**Prerequisites:** Complete `langgraph_101.ipynb`
<br>
<br>

---
<br>

> **Note:** These patterns are essential for production agents where safety, compliance, and user control are critical.


## Setup

Let's quickly set up our environment.

> **⚠️ Important Notes:**
> - **Before running this notebook**, make sure you've run `pnpm install` from the project root
> - **Wait a few seconds** between running cells to avoid tslab timing issues
> - If you get a "rebuildTimer" error, **restart the kernel** and try again


In [1]:
// Load environment variables
try {
    await import("dotenv/config");
    console.log("✓ Environment loaded successfully!");
} catch (error) {
    console.log("⚠️  Could not load dotenv. Make sure you've run 'pnpm install' from the project root.");
}

console.log("\n📝 Make sure OPENAI_API_KEY is set in your .env file or environment");


✓ Environment loaded successfully!

📝 Make sure OPENAI_API_KEY is set in your .env file or environment


In [2]:
import { initChatModel } from "langchain";

// Initialize model
const model = await initChatModel("openai:gpt-4o-mini");
console.log("✓ Model initialized!");


✓ Model initialized!


## Part 1: Human-in-the-Loop with Interrupts

### The Problem

Imagine you're building an agent that can send emails or make purchases. You don't want it to take these actions automatically - you want human approval first!

**Human-in-the-loop** lets you:
- Pause execution for review
- Approve, reject, or edit actions
- Add safety controls to sensitive operations

### How It Works

1. Agent encounters an `interrupt()` - execution pauses
2. System surfaces information to human
3. Human provides input (approve/reject/edit)
4. Agent resumes with `Command({ resume: ... })`


### Example 1: Simple Approval Workflow

Let's start with a simple example - asking for approval before sending an email.


In [3]:
import { tool } from "langchain";
import { z } from "zod";
import { interrupt } from "@langchain/langgraph";

const sendEmail = tool(
  async ({ to, subject, body }: { to: string; subject: string; body: string }) => {
    // Pause for human approval
    const approval = interrupt({
      action: "send_email",
      to: to,
      subject: subject,
      body: body,
      message: "Do you want to send this email?"
    });
    
    if (approval?.approved) {
      // In production, this would actually send the email
      return `✓ Email sent to ${to} with subject '${subject}'`;
    } else {
      return "Email cancelled by user";
    }
  },
  {
    name: "send_email",
    description: "Send an email to a recipient.",
    schema: z.object({
      to: z.string().describe("Email recipient"),
      subject: z.string().describe("Email subject"),
      body: z.string().describe("Email body")
    })
  }
);

// Test the tool directly
console.log("Tool created successfully!");
console.log(`Tool name: ${sendEmail.name}`);
console.log(`Tool description: ${sendEmail.description}`);


Tool created successfully!
Tool name: send_email
Tool description: Send an email to a recipient.


### Creating an Agent with Human-in-the-Loop

Now let's create an agent that uses this tool. **Remember:** Interrupts require a checkpointer!


In [4]:
import { createAgent } from "langchain";
import { MemorySaver } from "@langchain/langgraph";
import { v4 as uuidv4 } from "uuid";

// Create checkpointer for persistence
const checkpointer = new MemorySaver();

// Create agent with the email tool
const agent = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [sendEmail],
    systemPrompt: "You are a helpful email assistant. When asked to send emails, use the send_email tool.",
    checkpointer: checkpointer  // Required for interrupts
});

console.log("✓ Agent created with interrupt support!");


✓ Agent created with interrupt support!


### Running Until Interrupt

Let's run the agent and see it pause for approval:


In [5]:
import { HumanMessage } from "langchain";

// Create a unique thread for this conversation
const threadId = uuidv4();
const config = { configurable: { thread_id: threadId } };

// Run the agent and see it pause for approval
const result = await agent.invoke(
    {
        messages: [new HumanMessage("Send an email to alice@example.com with subject 'Meeting Tomorrow' and body 'Let's meet at 3pm.'")]
    },
    config
);

// Check if we hit an interrupt
if ("__interrupt__" in result) {
    console.log("Agent paused for approval\n");
    
    const interruptInfo = result.__interrupt__[0];
    const interruptValue = interruptInfo.value as { to: string; subject: string; body: string; message: string };
    
    console.log("Interrupt details:");
    console.log(`  To: ${interruptValue.to}`);
    console.log(`  Subject: ${interruptValue.subject}`);
    console.log(`  Body: ${interruptValue.body}`);
    console.log(`  Message: ${interruptValue.message}`);
} else {
    console.log("Agent completed without interrupt");
}


Agent paused for approval

Interrupt details:
  To: alice@example.com
  Subject: Meeting Tomorrow
  Body: Let's meet at 3pm.
  Message: Do you want to send this email?


### Resuming with Approval

Now let's approve the email and let the agent continue:


In [6]:
import { Command } from "@langchain/langgraph";

// Resume with approval
const result2 = await agent.invoke(
    new Command({ resume: { approved: true } }),
    config
);

// Print the final response
console.log("Final response:");
console.log(result2.messages[result2.messages.length - 1].content);


Final response:
The email has been sent to alice@example.com with the subject "Meeting Tomorrow" and the body "Let's meet at 3pm."


### Exercise: Try Rejecting the Email

Run the cells again, but this time reject the email by passing `{ approved: false }`:


In [7]:
// New thread for rejection example
const threadId2 = uuidv4();
const config2 = { configurable: { thread_id: threadId2 } };

// Run until interrupt
const result3 = await agent.invoke(
    {
        messages: [new HumanMessage("Send an email to bob@example.com saying 'Hello!'")]
    },
    config2
);

// Resume with rejection
const result4 = await agent.invoke(
    new Command({ resume: { approved: false } }),  // Reject the email
    config2
);

console.log("Final response:");
console.log(result4.messages[result4.messages.length - 1].content);


Final response:
It seems that the email was canceled. If you would like to send it again or modify it, please let me know!


## Part 2: Advanced Pattern - Edit Before Execution

Sometimes you want to **edit** the tool call, not just approve/reject it. Let's enhance our tool:


In [8]:
const sendEmailV2 = tool(
  async ({ to, subject, body }: { to: string; subject: string; body: string }) => {
    // Pause for human review
    const response = interrupt({
      action: "send_email",
      to: to,
      subject: subject,
      body: body,
      message: "Review this email. You can approve, reject, or edit it."
    });
    
    // Handle different response types
    if (response?.type === "approve") {
      return `Email sent to ${to} with subject '${subject}'`;
    } else if (response?.type === "reject") {
      return "Email cancelled";
    } else if (response?.type === "edit") {
      // Use edited values
      const editedTo = response.to || to;
      const editedSubject = response.subject || subject;
      const editedBody = response.body || body;
      return `Email sent with edits:\n  To: ${editedTo}\n  Subject: ${editedSubject}\n  Body: ${editedBody}`;
    }
    
    return "Unknown response";
  },
  {
    name: "send_email_v2",
    description: "Send an email to a recipient.",
    schema: z.object({
      to: z.string().describe("Email recipient"),
      subject: z.string().describe("Email subject"),
      body: z.string().describe("Email body")
    })
  }
);

// Create new agent with enhanced tool
const agentV2 = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [sendEmailV2],
    systemPrompt: "You are a helpful email assistant.",
    checkpointer: new MemorySaver()
});

console.log("✓ Agent V2 created with edit support!");


✓ Agent V2 created with edit support!


In [9]:
// Run and edit the email
const threadId3 = uuidv4();
const config3 = { configurable: { thread_id: threadId3 } };

// Run until interrupt
const result5 = await agentV2.invoke(
    {
        messages: [new HumanMessage("Send an email to team@example.com about the meeting")]
    },
    config3
);

console.log("Paused for review...\n");


Paused for review...



Now let's edit the email subject to make it URGENT meeting!


In [10]:
// Resume with edits
const result6 = await agentV2.invoke(
    new Command({
        resume: {
            type: "edit",
            subject: "URGENT: Meeting Today at 2pm",  // We have edited the email subject
            body: "This is the edited email body with more details."
        }
    }),
    config3
);

console.log("Final response:");
console.log(result6.messages[result6.messages.length - 1].content);


Final response:
The email has been sent to team@example.com with the subject "URGENT: Meeting Today at 2pm" and included more details in the body. If you need anything else, feel free to ask!


## Part 3: Middleware Approach to Human-in-the-Loop

Instead of manually adding `interrupt()` calls to each tool, we can use **middleware** to add human-in-the-loop to any tool automatically!

This is cleaner and more maintainable for production systems.


In [11]:
import { humanInTheLoopMiddleware } from "langchain";

// Simple email tool without manual interrupt
const simpleEmailTool = tool(
  async ({ to, subject, body }: { to: string; subject: string; body: string }) => {
    // No interrupt here - middleware handles it!
    return `Email sent to ${to} with subject '${subject}'`;
  },
  {
    name: "send_email",
    description: "Send an email to a recipient.",
    schema: z.object({
      to: z.string().describe("Email recipient"),
      subject: z.string().describe("Email subject"),
      body: z.string().describe("Email body")
    })
  }
);

// Create agent with humanInTheLoopMiddleware
const agentWithMiddleware = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [simpleEmailTool],
    systemPrompt: "You are a helpful email assistant.",
    middleware: [
        humanInTheLoopMiddleware({
            interruptOn: {
                send_email: true  // Require approval for this tool
            },
            descriptionPrefix: "Tool execution pending approval"
        })
    ],
    checkpointer: new MemorySaver()
});

console.log("✓ Agent created with humanInTheLoopMiddleware!");


✓ Agent created with humanInTheLoopMiddleware!


In [12]:
// Test the middleware approach
const threadId4 = uuidv4();
const config4 = { configurable: { thread_id: threadId4 } };

// Run until interrupt
const result7 = await agentWithMiddleware.invoke(
    {
        messages: [new HumanMessage("Send an email to charlie@example.com saying 'Great work!'")] 
    },
    config4
);

if ("__interrupt__" in result7) {
    console.log("Agent paused for approval via middleware\n");
    const interruptInfo = result7.__interrupt__[0];
    console.log("Interrupt value:", JSON.stringify(interruptInfo.value, null, 2));
}


Agent paused for approval via middleware

Interrupt value: {
  "actionRequests": [
    {
      "name": "send_email",
      "args": {
        "to": "charlie@example.com",
        "subject": "Feedback",
        "body": "Great work!"
      },
      "description": "Tool execution pending approval\n\nTool: send_email\nArgs: {\n  \"to\": \"charlie@example.com\",\n  \"subject\": \"Feedback\",\n  \"body\": \"Great work!\"\n}"
    }
  ],
  "reviewConfigs": [
    {
      "actionName": "send_email",
      "allowedDecisions": [
        "approve",
        "edit",
        "reject"
      ]
    }
  ]
}


In [13]:
// Resume with approval using the middleware format
const result8 = await agentWithMiddleware.invoke(
    new Command({ 
        resume: { 
            decisions: [{ type: "approve" }] 
        } 
    }),
    config4
);

console.log("Final response:");
console.log(result8.messages[result8.messages.length - 1].content);


Final response:
The email saying "Great work!" has been successfully sent to charlie@example.com.


## Part 4: Custom Middleware

**Middleware** provides fine-grained control over the agent loop. It lets you:
- Inspect state before/after model calls
- Modify model requests dynamically
- Add custom logic at key execution points

### The Agent Loop

```
Input --> [beforeModel] --> [wrapModelCall] --> Model --> [afterModel] --> Tools --> ...
```

Middleware hooks into this loop:
- **`beforeModel`** - Runs before model execution, can update state
- **`wrapModelCall`** - Wraps the model call, can modify request/response
- **`afterModel`** - Runs after model execution, before tools


### Example 1: Dynamic System Prompt

Let's create middleware that changes the system prompt based on the user's role:


In [14]:
import { createMiddleware } from "langchain";

// Define context schema
const contextSchema = z.object({
    userRole: z.enum(["beginner", "expert"]).default("beginner")
});

// Create middleware using createMiddleware
const dynamicPromptMiddleware = createMiddleware({
    name: "DynamicPrompt",
    wrapModelCall: (request, handler) => {
        const userRole = request.runtime.context?.userRole || "beginner";
        
        let systemPrompt;
        if (userRole === "expert") {
            systemPrompt = "You are an AI assistant for experts. Provide detailed technical responses with code examples.";
        } else if (userRole === "beginner") {
            systemPrompt = "You are an AI assistant for beginners. Explain concepts simply, avoid jargon.";
        } else {
            systemPrompt = "You are a helpful AI assistant.";
        }
        
        return handler({ ...request, systemPrompt });
    }
});

console.log("✓ Dynamic prompt middleware created!");


✓ Dynamic prompt middleware created!


In [15]:
const explainConcept = tool(
  async ({ concept }: { concept: string }) => {
    const explanations: Record<string, string> = {
      "async": "Asynchronous programming allows code to run without blocking.",
      "recursion": "Recursion is when a function calls itself."
    };
    return explanations[concept.toLowerCase()] || "Concept not found.";
  },
  {
    name: "explain_concept",
    description: "Explain a programming concept.",
    schema: z.object({
      concept: z.string().describe("The concept to explain")
    })
  }
);

// Create agent with middleware
const agentWithDynamicPrompt = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [explainConcept],
    middleware: [dynamicPromptMiddleware],
    contextSchema: contextSchema
});

console.log("✓ Agent with dynamic prompt middleware created!");


✓ Agent with dynamic prompt middleware created!


### Testing Different User Roles

Let's see how the agent responds differently based on user role:


In [16]:
// Expert user
console.log("=".repeat(50));
console.log("EXPERT USER");
console.log("=".repeat(50));

const expertResult = await agentWithDynamicPrompt.invoke(
    { messages: [new HumanMessage("Explain async programming")] },
    { context: { userRole: "expert" } }
);
console.log(expertResult.messages[expertResult.messages.length - 1].content);
console.log();

// Beginner user
console.log("=".repeat(50));
console.log("BEGINNER USER");
console.log("=".repeat(50));

const beginnerResult = await agentWithDynamicPrompt.invoke(
    { messages: [new HumanMessage("Explain async programming")] },
    { context: { userRole: "beginner" } }
);
console.log(beginnerResult.messages[beginnerResult.messages.length - 1].content);


EXPERT USER
It seems that I am unable to retrieve a predefined explanation for asynchronous programming. However, I can provide a detailed explanation along with code examples.

### Asynchronous Programming

Asynchronous programming is a programming paradigm that allows a program to perform tasks concurrently without blocking the execution of other tasks. This is particularly useful for I/O-bound operations, like network requests or file reading, where the program can continue executing while waiting for a response.

#### Key Concepts

1. **Asynchronous Functions**: Functions that can operate asynchronously. They allow the execution of multiple tasks simultaneously.

2. **Callbacks**: Functions that are passed as arguments to other functions and are executed after a specific event or task is completed.

3. **Promises**: Objects that represent the eventual completion (or failure) of an asynchronous operation and its resulting value. They can be in one of three states: pending, fulfilled

### Example 2: Custom Middleware - Request Logger

Middleware lets you hook into the agent loop and see what's happening at each step. This is incredibly useful for debugging and understanding how your agent works.

**The Agent Loop:**
```
User Input --> [beforeModel] --> [wrapModelCall] --> Model --> [afterModel] --> Tools --> ...
```

**What we'll build:**
A logger that prints information at each step:
- **Before model** - How many messages are in the conversation?
- **After model** - Did the model call a tool or give a final answer?

This is like adding debug `console.log()` statements, but in a clean, reusable way!

Let's create middleware that logs model requests for debugging:


In [17]:
const requestLoggerMiddleware = createMiddleware({
    name: "RequestLogger",
    beforeModel: (state) => {
        console.log(`[BEFORE MODEL] Processing ${state.messages.length} messages`);
        return; // Don't modify state
    },
    afterModel: (state) => {
        const lastMessage = state.messages[state.messages.length - 1];
        if (lastMessage.additional_kwargs?.tool_calls && lastMessage.additional_kwargs.tool_calls.length > 0) {
            console.log(`[AFTER MODEL] Model requested ${lastMessage.additional_kwargs.tool_calls.length} tool call(s)`);
        } else {
            console.log(`[AFTER MODEL] Model provided final response`);
        }
        return; // Don't modify state
    }
});

// Create agent with logger middleware
const agentWithLogger = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [explainConcept],
    middleware: [requestLoggerMiddleware]
});

console.log("✓ Agent with logger middleware created!");


✓ Agent with logger middleware created!


### What to Expect

When we run the agent with the logger, you'll see the execution flow in real-time:

**First iteration:**
1. `[BEFORE MODEL]` - Shows how many messages we're starting with
2. `[AFTER MODEL]` - The model decides to call the `explain_concept` tool

**Second iteration (after tool execution):**
1. `[BEFORE MODEL]` - Now we have more messages (including tool result)
2. `[AFTER MODEL]` - Model provides the final answer (no more tools needed)

This gives you a detailed view into your agent's decision-making process!

Let's run it:


In [18]:
// Run and observe the logs
console.log("\n" + "=".repeat(50));
console.log("RUNNING AGENT WITH LOGGER");
console.log("=".repeat(50) + "\n");

const loggerResult = await agentWithLogger.invoke({
    messages: [{ role: "user", content: "Explain recursion" }]
});

console.log("\n" + "=".repeat(50));
console.log("FINAL RESPONSE");
console.log("=".repeat(50));
console.log(loggerResult.messages[loggerResult.messages.length - 1].content);



RUNNING AGENT WITH LOGGER

[BEFORE MODEL] Processing 1 messages
[AFTER MODEL] Model requested 1 tool call(s)
[BEFORE MODEL] Processing 3 messages
[AFTER MODEL] Model provided final response

FINAL RESPONSE
Recursion is a programming concept where a function solves a problem by calling itself with a smaller or simpler input. This self-referential approach allows complex problems to be broken down into simpler subproblems. A recursive function typically has two main components:

1. **Base Case**: This is the condition under which the recursion stops. It defines the simplest case that can be solved directly without further recursion.

2. **Recursive Case**: This is where the function calls itself with modified arguments, moving closer to the base case.

Recursion can be very useful for tasks such as traversing data structures (like trees), implementing algorithms (like quicksort or mergesort), and solving mathematical problems (such as calculating factorials or Fibonacci numbers). Howeve

## Part 5: Combining Middleware and Human-in-the-Loop

Let's combine human-in-the-loop AND middleware for a production-ready agent with multiple layers of safety:


In [19]:
// Sensitive tool that needs approval
const deleteDatabase = tool(
  async ({ databaseName }: { databaseName: string }) => {
    const response = interrupt({
      action: "delete_database",
      database_name: databaseName,
      warning: "This will permanently delete the database!",
      message: "Are you absolutely sure?"
    });
    
    if (response?.confirmed) {
      return `Database '${databaseName}' has been deleted (simulation)`;
    } else {
      return "Database deletion cancelled";
    }
  },
  {
    name: "delete_database",
    description: "Delete a database. THIS IS DANGEROUS!",
    schema: z.object({
      databaseName: z.string().describe("Name of the database to delete")
    })
  }
);

// Middleware to track dangerous operations
const safetyMiddleware = createMiddleware({
    name: "SafetyChecker",
    afterModel: (state) => {
        const lastMessage = state.messages[state.messages.length - 1];
        
        if (lastMessage.additional_kwargs?.tool_calls) {
            for (const toolCall of lastMessage.additional_kwargs.tool_calls) {
                if (toolCall.function.name.toLowerCase().includes("delete")) {
                    console.log("⚠️  [SAFETY] Dangerous operation detected!");
                    console.log(`   Tool: ${toolCall.function.name}`);
                    console.log(`   Args: ${toolCall.function.arguments}`);
                }
            }
        }
        
        return;
    }
});

// Create production agent with both safety layers
const productionAgent = createAgent({
    model: "openai:gpt-4o-mini",
    tools: [deleteDatabase],
    middleware: [safetyMiddleware],
    checkpointer: new MemorySaver()
});

console.log("✓ Production agent with safety middleware created!");


✓ Production agent with safety middleware created!


### What to Expect: Layered Safety in Action

When we attempt a dangerous operation, you'll see **both** safety mechanisms activate:

**Layer 1 - Middleware Detection:**
- `[SAFETY] Dangerous operation detected!` - Middleware spots the delete operation
- Logs the tool name and arguments for audit trails

**Layer 2 - Human Approval (Interrupt):**
- Agent execution pauses at the `interrupt()`
- Warning message displayed to human reviewer
- Execution won't continue until explicit approval

**This is defense-in-depth:** Middleware monitors ALL operations, while interrupts enforce human approval for critical actions.


In [20]:
// Test the combined pattern
const threadId5 = uuidv4();
const config5 = { configurable: { thread_id: threadId5 } };

console.log("\n" + "=".repeat(50));
console.log("DANGEROUS OPERATION ATTEMPT");
console.log("=".repeat(50) + "\n");

// Run until interrupt
const dangerResult = await productionAgent.invoke(
    {
        messages: [new HumanMessage("Delete the production_db database")]
    },
    config5
);

if ("__interrupt__" in dangerResult) {
    const interruptInfo = dangerResult.__interrupt__[0];
    const interruptValue = interruptInfo.value as { warning: string; database_name: string };
    console.log("\n🛑 Human approval required:");
    console.log(`   ${interruptValue.warning}`);
    console.log(`   Database: ${interruptValue.database_name}`);
}

console.log("\n(In a real app, a human would review this before proceeding)");



DANGEROUS OPERATION ATTEMPT

⚠️  [SAFETY] Dangerous operation detected!
   Tool: delete_database
   Args: {"databaseName":"production_db"}

🛑 Human approval required:
   This will permanently delete the database!
   Database: production_db

(In a real app, a human would review this before proceeding)


## Key Takeaways

### Human-in-the-Loop (Interrupts)
- ✓ Use `interrupt()` to pause execution
- ✓ Requires a `checkpointer` for persistence
- ✓ Resume with `new Command({ resume: value })`
- ✓ Perfect for approval workflows and sensitive operations

### Middleware
- ✓ `wrapModelCall` - Adjust prompts, models, tools dynamically
- ✓ `beforeModel` / `afterModel` - Add custom logic at key points
- ✓ Use `createMiddleware()` for reusable components
- ✓ Perfect for logging, safety checks, dynamic behavior

### When to Use What?

**Use Manual Interrupts when:**
- You need custom approval logic per tool
- You want to support edit/approve/reject
- You need fine-grained control over the interrupt payload

**Use humanInTheLoopMiddleware when:**
- You want consistent approval workflows across tools
- You need to add HITL to existing tools without modifying them
- You want a declarative configuration approach

**Use Custom Middleware when:**
- You need to modify agent behavior dynamically
- You want to add logging/monitoring
- You need to enforce policies (token limits, safety checks)
- You want to personalize responses based on context


## Practice Exercise (Optional)

Try building an agent that:
1. Has a tool to make a purchase
2. Uses middleware to check if the purchase amount is over $1000
3. If over $1000, uses an interrupt to require approval
4. If under $1000, processes automatically

Hint: Combine `beforeModel` or `afterModel` middleware with conditional `interrupt()` logic!


In [21]:
// Your code here!
// Challenge: Build the purchase approval agent

// const makePurchase = tool(
//   async ({ item, amount }: { item: string; amount: number }) => {
//     ...
//   },
//   { ... }
// );

// const purchaseLimitMiddleware = createMiddleware({
//   ...
// });


## Next Steps

You now have powerful tools for building production agents!

**Continue your journey:**
1. ✓ Check out `multi_agent.ipynb` for multi-agent systems
2. ✓ Explore built-in middleware (Summarization, PII Redaction)
3. ✓ Build your own custom middleware for your use case
4. ✓ Add LangSmith for debugging and monitoring

**Resources:**
- [Middleware Documentation](https://docs.langchain.com/oss/javascript/langchain/middleware)
- [Human-in-the-Loop Guide](https://docs.langchain.com/oss/javascript/langchain/human-in-the-loop)
- [LangGraph Documentation](https://docs.langchain.com/oss/javascript/langgraph/overview)
<br>
<br>

**Happy building!**
